# Basics

## A Very Simple Example

A simple fortran code consists of a _main program_ containing a series of instructions. Modern versions typically have a file extension `.f90`, which signifies free format source. The `.f` extension indicates old-style fixed format.
The use of the `!` symbol tells the compiler than any subsequent text on that line is a _comment_. Comments can have a line to themselves or they can follow code and it's always a good idea to use comments to make sure your code is well documented. **However**, don't hide code within comments; keep the two easily distinguishable to make the source easy to read.

In [ ]:
%num_images: 1
Program hello              ! The name of the program is a matter of personal preference. 

  ! The Implict none command tells the compiler to treat undeclared variables as errors.
  ! It's strongly advised to used this as it catches spelling mistakes and forces the
  ! programmer to give each variable a type.
  Implicit none
  
  Integer     :: myNumber  ! Declare the variable myNumber to be of type 'integer'
  
  myNumber = 25            ! Assign a value to a variable
  
  Print *, 'Hello World!'  ! Comments can also follow Fortran instructions
  Print *, 'I like the number', myNumber

End program                ! The program must terminate with this line.

### Compiling the Code

The above code has to be compiled before it can be run, converting it from (near) English to machine language. Using the _GNU Fortran_ compiler and assuming the file containing the source was named `hello.f90`, this could be achieved as follows:

```bash
gfortran -o hello hello.f90
```

This will produce an executable file called `hello`, which can be executed on Linux:

```bash
./hello
```

### Code Description

The above example, while basic, demonstrates some of the fundamental concepts that are applicable to any Fortran program:

 * Units or sections (in this case the _program_).
 * Variables, used to hold information.
  * Type.
  * Declaration.
  * Assignment.
  * De-referencing.
 * Output

Each of these topics will be discussed in further detail, but will be familiar to those with experience in other programming languages.

### Exercise 1 - _Hello World!_

Copy the above code into a text file named `ex1.f90`, using a text editor such as `nano`.
 * Compile and run it.
 * Change the output produced by the program. Does it still run?

## Variables

Fortran traditionally has used _implicit variable declaration_, whereby the variable name and type don't have to be fixed ahead of use. The _type_ of the variable was determined by the first character of the name and the variable name length was limited to 6 characters. This led to variables such as `ievnct`, representing an _integer_ used to hold an _event counter_. Needless to say, this made Fortran 77 and earlier versions hard to read and debug! Fortunately this has changed in Modern Fortran.

### Variable declaration

In Modern Fortran, variables should be declared before use, using the following form:

```fortran
TYPE(kind)<attributes>   :: varName
```

In practice, this results in declarations such as the following:

```fortran
Integer                  :: varName
```

The optional `attributes` list allows the author to specify additional conditions, such as that is is a read-only `parameter`. The optional `kind` parameter specifies the internal representation of the variable, which in practice means precision and range. Both these options will be covered in more depth.

### Intrinsic Types

The five basic (or intrinsic) data types supported by Fortran are as follows:

| Type          | Example       | Description                                 |
| ------------- | :------------ | :------------------------------------------ |
| **INTEGER**   | `-25`         | Signed integer                              |
| **REAL**      | `7.2e-3`      | Signed floating point                       |
| **COMPLEX**   | `(1.0, 2.5)`  | Complex number                              |
| **LOGICAL**   | `.true.`      | Boolean `.true.` or `.false.`               |
| **CHARACTER** | `'A string'`  | Generally used as an array to form a string |

Each of these types has a default size, or _kind_.

**NOTE**: some compilers offer the `DOUBLE PRECISION` type or allow setting the size with the `kind=(x)`, where `x` is the number of bytes requested, but _these are not parts of standard Fortran_ and should be avoided  when writing portable code. Additionally, compilers allow changing the default size of the intrinsic types, so if you want to be sure that your code will run as expected on any architecture it's important to specify the size of every variable you declare.

#### Specifying Intrinsic Type Storage Size

Until recently this was a problem. However, recent Fortran standards introduced options. In Fortran 2003 it is possible to use the `selected_real_kind` intrinsic to define variable size:

In [ ]:
%num_images: 1
Program kind_demo
  Implicit none
  
  ! What is the smallest number of bytes required to store a real value with
  ! 15 digits of precision and an expotent range of 307?
  Integer, parameter   :: dp = selected_real_kind(p=15, r=307)
 
  ! Use this size to declare a Real variable that is stored using the 
  ! computed number of bytes.
  Real(kind=dp)        :: myLongVar
  
  ! Assign it a value too large for a single precision real to check.
  myLongVar = 2.0d200
  
  ! Print out the value
  print *, 'myLongVar = ', myLongVar

End program     

Fortran 2008 and later contain predefined constants representing the precision and range of different sizes of real variables, and this is the recommended method of declaring them if available:

In [ ]:
%num_images: 1
Program iso_env
  ! The iso_fortran_env module contains a number of definitions of ISO approved values
  use, intrinsic  :: iso_fortran_env 
  
  Implicit none

  ! REAL64 is defined in the above module and is the same value as obtained
  ! from using the selected_real_kind intrinsic.
  Integer, parameter  :: dp = REAL64
  
  Real(kind=dp)       :: myLongVar
  
  myLongVar = 2.0d200
  
  print *, 'myLongVar = ', myLongVar

End program

### Exercise 2 - _Variables_

Fortran uses a number of familiar mathematical operators to perform arithmetical operations.
 * Write a short program to declare variables of different types and try adding, subtracting, multiplying and dividing one by another.
 * In the above examples the exponential form of a real value is written in the form `2.0d200`, where **d** is short for _double precision exponential_. The standard form in Fortran is `2.0e200`, where **e** is for exponential. What happens when you change the code to that form? Can you explain the result?

## Arrays

Arrays in Fortran are collections of variables stored under a single identifier. An array can have multiple _dimensions_, allowing the creations of vectors, matrices, and higher order collections. Unlike more generic programming languages such a Python, every array element must be of the same type. So, whereas in Python the following is a perfectly valid _list_:

```python
my_list = []
my_list = [12, 34.5, "what is this string doing here?", True]
```

...this would not be possible in Fortran. Whilst this may seem to be a loss of functionality, the big advantage of having a fixed data type is speed.

### Array Declaration

Declaring an array is similar to declaring a variable, but we must also specify the _dimensions_ and _size_ of each dimension. Following from the variable declaration in the previous section, an array declaration would look like this:

```fortran
TYPE(kind)<attributes>   :: arrName(xsize, ysize, ...)
```

The big difference is that after the array name we have to specify the size of each dimension, separated by a comma. For a 3X5 matrix, the declaration would look like this:

```fortran
Integer                  :: arrName(3, 5)
```

Unlike a Python list, we can't elect to leave the size empty and add elements as we go along. Again, this is a sacrifice of convenience in the name of speed. We'll see a way of getting around this later in the course, but for the moment assume that arrays are of fixed size and dimentionality.

Assigning values to or retrieving values from an array is achieved by specifying the array element in brackets after the array name. For example:

In [ ]:
%num_images: 1
Program my_array
  ! Declare a two dimentsional array.
  Integer  :: arrName(3,5)
  
  ! Assign a value to one element of this array.
  arrName(1,2) = 1000
  
  ! Print this value out to make sure it has been assigned correctly.
  print *, 'arrName(1,2) = ', arrName(1,2)

End program

Try modifying this example to assign additional values.

### Whole Array Assignment

As we've seen, it is possible to allocate array elements individually, but this can be inconvenient when you have a lot of elements to populate. We've also seen that it's not safe for elements to remain without allocated values as this will lead to unpredictable behaviour. In Modern Fortran it is possible to allocate a number of elements in one go:

In [ ]:
%num_images: 1
Program assign_elements
  ! Declare a one dimentsional array.
  Integer  :: arrName(6)
  
  ! Assign all elements at once.
  arrName = [1, 100, 50, 75, 200, 300]
  
  ! Print out one of the values to check it has been correctly assigned.
  print *, 'arrName(4) = ', arrName(4)

End program

Could we do this with arrays of more than one dimension? Not directly, but Fortran provides _intrinsic functions_ for manipulating arrays, so we could convert a one-dimensional array into a two-dimensional array in a single step:

In [ ]:
%num_images: 1
Program reshape_array
  ! Declare a two dimentsional array.
  Integer  :: arrName(2, 3)
  
  ! Shape a list of values to the dimensions of the array.
  arrName = reshape([1, 100, 50, 75, 200, 300], shape(arrName))
  
  ! Print out some elements of the two dimensional array.
  print *, 'arrName(1, 2) = ', arrName(1, 2)
  print *, 'arrName(2, 1) = ', arrName(2, 1)

End program

This may not produce the results you were expecting. To explain it we need to know that, unlike many familiar langages such as C++, Fortran stores arrays in _column-major_ order. This means that a (2, 3) array looks like this:

|[]()|[]()|[]()|
| -- | -- | -- |
| x  | x  | x  |
| x  | x  | x  |

When allocated, columns are filled first, then rows, so the fully populated array looks like this:

|[]() |[]() |[]() |
| --- | --- | --- |
| 1   | 50  | 200 |
| 100 | 75  | 300 |

This has consequences when mixing C++ and Fortran code. Arrays must be _transposed_ when passed between functions, which takes time and memory. We'll take a look at this later on, but for the moment simply remember that Fortran arrays must be read column first.

### Exercise 3 - _Arrays_

 * Putting together the information found in the _variables_ and _arrays_ sections, can you write a program that creates an array of floating point values in a portable manner?
 * What happens if you try and print out an array element that hasn't been allocated? Should this be a concern?
 * What happens if you try and allocate a value outside of the array dimensions?
 * Ensure you know how multi-dimensional arrays work in Fortran by creating one and then assigning values to it in a single step.
 * Construct a floating-point array and print out some of its values.